# Topic modelling of news headlines for prediction of news category with 6 topics

Now we wan't to perform topic modelling with unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier and assume 6 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

import wordcloud

import re

import nlp_preprocessors

import pyLDAvis.lda_model

## Load train set

In [2]:
train = pd.read_csv('../../data/01_train_nosplit_preprocessed.csv')

In [3]:
train.head()

,date,title,description,category,text,source,title_description_text
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...


## Load German stopwords

In [4]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [5]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

In [6]:
# append words to stopwords that contain no information
stop_words += ['wegen', 'melden', 'meldet', 'können', 'könnte', 'könnten', 'update', 'neu', 'neue', 'neues', 'ohne', 'letzte', 'letzter',
                  'letztes', 'eins', 'zwei', 'drei', 'vier', 'fünf', 'sechs', 'sieben', 'acht', 'neun', 'zehn',
                   'gehen', 'geht', 'wollen', 'wollte', 'wollt', 'jahr', 'jahre', 'fordern', 'fordert',
                  'warnen', 'warnt', 'frühjahr', 'frühling', 'sommer', 'herbst', 'winter', 'erneut', 'deutlich', 'schwer', 'jahren', 'woche',
                  'wochen', 'monat', 'monate', 'tag', 'tage', 'stunden', 'stunde', 'minuten', 'minuten', 'ende', 'beenden', 'endet']

## Clean text

In [7]:
train['title_cleaned'] = train['title'].apply(lambda x: nlp_preprocessors.clean_text(x))

In [8]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt


## Extract features from 'title' 

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)

In [11]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['title_cleaned'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [12]:
dtm

<67513x21412 sparse matrix of type '<class 'numpy.int64'>'
	with 330201 stored elements in Compressed Sparse Row format>

In [13]:
from sklearn.decomposition import LatentDirichletAllocation

In [14]:
lda = LatentDirichletAllocation(n_components=6, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=6, random_state=42)

## Analyse extracted features

In [15]:
len(cv.get_feature_names_out())

21412

In [16]:
cv.get_feature_names_out()

array(['000', '034', '039', ..., 'войны', 'российские', 'украинцы'],
      dtype=object)

In [17]:
len(lda.components_)

6

In [18]:
lda.components_

array([[ 0.1685613 , 17.07200623,  7.07743311, ...,  0.16666855,
         0.16666855,  0.16667093],
       [ 3.01553633,  5.54811027,  0.1677516 , ...,  2.16665594,
         2.16665594,  0.16667167],
       [13.31585468,  8.61638762, 28.02462403, ...,  0.16666873,
         0.16666873,  0.16667134],
       [ 0.16666811, 35.90896048,  4.58918905, ...,  0.16666893,
         0.16666893,  0.16667179],
       [ 0.16671151, 16.28712606, 16.97407526, ...,  0.16666897,
         0.16666897,  2.16664261],
       [ 0.16666807,  6.56740935,  0.16692695, ...,  0.16666887,
         0.16666887,  0.16667165]])

In [19]:
len(lda.components_[0])

21412

### Show most important words of first extracted topic

In [20]:
first_topic = lda.components_[0]

In [21]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([19585,  3800, 20969, ..., 15087, 10150, 18325], dtype=int64)

In [22]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [23]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

liveblog
selenskyj
kiew
angriff
lage
putin
russland
russisch
krieg
ukraine


### Show most important words of first extracted topic

In [24]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['moskau', 'chef', 'stimme', 'invasion', 'ukrainisch', 'liveblog', 'selenskyj', 'kiew', 'angriff', 'lage', 'putin', 'russland', 'russisch', 'krieg', 'ukraine']


Die TOP-15 Wörter für das Thema #1
['großbritannien', 'panzer', 'kritik', 'faeser', 'spd', 'land', 'fdp', 'wahrheit', 'habeck', 'lindner', 'streit', 'eu', 'regierung', 'ampel', 'euro']


Die TOP-15 Wörter für das Thema #2
['wahl', 'eu', 'marktbericht', 'welt', 'taiwan', 'groß', 'analyse', 'wm', 'biden', 'usa', 'berlin', 'scholz', 'us', 'deutschland', 'china']


Die TOP-15 Wörter für das Thema #3
['brauchen', 'ex', 'groß', 'deutschland', 'afd', 'g7', 'frankreich', 'klima', 'türkei', 'gipfel', 'wahl', 'israel', 'russland', 'putin', 'deutsch']


Die TOP-15 Wörter für das Thema #4
['europa', 'gas', 'mann', 'union', 'protest', 'sanktion', 'grüne', 'gastbeitrag', 'cdu', 'bundesliga', 'frau', 'tote', 'iran', 'bayern', 'eu']


Die TOP-15 Wörter für das Thema #5
['präsident', 'mensch', 'steigen', 'new

### Get topics with highest probability for news items in train data

In [25]:
topic_results = lda.transform(dtm)
topic_results.shape

(67513, 6)

In [26]:
topic_results[0].round(5)

array([0.02085, 0.25909, 0.02091, 0.30855, 0.22744, 0.16315])

In [27]:
topic_results[0].argmax()

3

In [28]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [29]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned,topic
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,3
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,0
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,1
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben,0
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt,1


## Visualize model

In [30]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(lda, dtm, cv, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      -56.170578   82.302704       1        1  18.854537
2       78.270287   73.158676       2        1  17.112012
1     -106.015251  -43.265148       3        1  16.442319
5       -2.118897 -129.247849       4        1  16.124677
3      111.553162  -56.989056       5        1  15.944318
4        4.911771  -14.853953       6        1  15.522137, topic_info=           Term         Freq        Total Category  logprob  loglift
18325   ukraine  4538.000000  4538.000000  Default  30.0000  30.0000
10150     krieg  2629.000000  2629.000000  Default  29.0000  29.0000
15087  russisch  1381.000000  1381.000000  Default  28.0000  28.0000
3583     corona  1142.000000  1142.000000  Default  27.0000  27.0000
15091  russland  1763.000000  1763.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
6369    fußball   151.355463   337.772023   Topic6  -5.8316   1.0602
10650     leben   150.508471   358.715052   Topic6  -5.8372   0.9944
16977   sterben   146.624834   479.683944   Topic6  -5.8634   0.6777
7647     habeck   149.202094   712.313451   Topic6  -5.8459   0.2997
2314     berlin   150.282453   981.855624   Topic6  -5.8387  -0.0140

[374 rows x 6 columns], token_table=       Topic      Freq              Term
term                                    
333        2  0.981454        abtreibung
339        2  0.988613  abtreibungsrecht
418        2  0.161893               afd
418        3  0.180838               afd
418        5  0.502902               afd
...      ...       ...               ...
20777      3  0.081953            zahlen
21262      3  0.959805        österreich
21262      5  0.034776        österreich
21275      1  0.993151         überblick
21368      6  0.990083    überschwemmung

[681 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 6, 4, 5])

## Save train set with topics and dump model

In [31]:
#Save result to csv
train.to_csv('train_lda_topics_6.csv')

In [32]:
# Dump countvectorizer
joblib.dump(cv, 'models/cv.jl')

['models/cv.jl']

In [33]:
# Dump LDA model
joblib.dump(lda, 'models/lda_model_6_topics.jl')

['models/lda_model_6_topics.jl']